In [1]:
import re
from collections import Counter
import numpy as np
import random
import sys
import math

In [2]:
with open('Ulysses - James Joyce.txt', 'r') as file:
    data = file.readlines()
    #print(data)


In [3]:
while ("\n" in data):
    data.remove("\n")
    

In [4]:

def tokenization(text):
    # Define the regular expression patterns to match words, punctuation, URLs, hashtags, and mentions
    word_pattern = r'\b\w+\b'
    punctuation_pattern = r'[^\w\s]'
    url_pattern = r'(?:http[s]?://|www\.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    hashtag_pattern = r'#\w+[\w\d]+'

    mention_pattern = r'@\w+[\w\d]+'
    percentage_pattern=r"^\d+%|\s\d+%"
    
    # Use the re.findall() function to match the patterns in the text
    words = re.findall(word_pattern, text)
    punctuations = re.findall(punctuation_pattern, text)
    urls = re.findall(url_pattern, text)
    hashtags = re.findall(hashtag_pattern, text)
    mentions = re.findall(mention_pattern, text)
    percentage= re.findall(percentage_pattern, text)
    
    text = text.replace('\n'," ")
    text = text.strip()
    
    # Replace the tokens with appropriate placeholders
    text = re.sub(url_pattern, "<URL>", text)
    text = re.sub(hashtag_pattern, "<HASHTAG>", text)
    text = re.sub(mention_pattern, "<MENTION>", text)
    text = re.sub(r'(\W)\1{1,}', r'\1', text)
    text = re.sub(r"^\d+[^\s]*|\s\d+[^\s]*"," <NUMEXP>",text)

    
    return text

In [5]:
import string
def remove_punchu(text):
    punctuations = string.punctuation
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    return text


        

In [6]:
def add_bracket(text):
    

    text= text.replace("URL","<URL>")
    text= text.replace("HASHTAG","<HASHTAG>")
    text= text.replace("MENTION","<MENTION>")
    text= text.replace("NUMEXP","<NUMEXP>")
    
    return text.lower()

In [7]:
new_data=[]
for sub_data in data:
    sub_data=tokenization(sub_data)
    sub_data=remove_punchu(sub_data)
    sub_data=add_bracket(sub_data)
    new_data.append(sub_data)
    
new_data

['the project gutenberg ebook of ulysses by james joyce',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 '<url> if you are not located in the united states you',
 'will have to check the laws of the country where you are located before',
 'using this ebook',
 'title ulysses',
 'author james joyce',
 'release date december <numexp> <numexp> ebook <hashtag>',
 'most recently updated december <numexp> <numexp>',
 'language english',
 'character set encoding utf8',
 'produced by col choat and david widger',
 ' start of the project gutenberg ebook ulysses ',
 'illustration',
 'ulysses',
 'by james joyce',
 'contents',
 '— i —',
 ' <numexp> ',
 ' <numexp> ',
 ' <numexp> ',
 '— ii —',
 ' <numexp> ',
 ' <numexp> ',
 ' <numexp> ',
 ' <numexp> ',
 

In [8]:


with open("tokenization.txt", "w") as f:
    for item in new_data:
        f.write(item + "\n")
    

In [9]:
new_data


['the project gutenberg ebook of ulysses by james joyce',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 '<url> if you are not located in the united states you',
 'will have to check the laws of the country where you are located before',
 'using this ebook',
 'title ulysses',
 'author james joyce',
 'release date december <numexp> <numexp> ebook <hashtag>',
 'most recently updated december <numexp> <numexp>',
 'language english',
 'character set encoding utf8',
 'produced by col choat and david widger',
 ' start of the project gutenberg ebook ulysses ',
 'illustration',
 'ulysses',
 'by james joyce',
 'contents',
 '— i —',
 ' <numexp> ',
 ' <numexp> ',
 ' <numexp> ',
 '— ii —',
 ' <numexp> ',
 ' <numexp> ',
 ' <numexp> ',
 ' <numexp> ',
 

In [10]:
def split_text(text):
    text=text.split(" ")
    words=[]
    for word in text:
        words.append(word)
    return words




In [11]:
word_token=[] 
for line in new_data:
    line='<s> <s> <s> '+line+" </s>"
    word_token.append(split_text(line))
    
    
    


In [12]:
word_token[1]

['<s>',
 '<s>',
 '<s>',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'in',
 'the',
 'united',
 'states',
 'and',
 '</s>']

In [13]:
len(word_token)

25797

In [14]:
test_set=word_token[-1000:]
train_set=word_token[:-1000]


In [15]:
def replace_unk(data):
    word_freq = {word: sum(word in sentence for sentence in data) for sentence in data for word in sentence}
    return [[word if word_freq[word] >= 3 else '<unk>' for word in sentence] for sentence in data]


In [16]:
from collections import defaultdict

def replace_unk(data):
    word_freq = defaultdict(int)
    for sentence in data:
        for word in set(sentence):
            word_freq[word] += 1

    return [[word if word_freq[word] >= 2 else '<unk>' for word in sentence] for sentence in data]


In [17]:
train_set=replace_unk(train_set)


In [18]:
print(len(train_set))
len(test_set)

24797


1000

In [19]:
from collections import defaultdict
max_n = 4
ngrams_counts = defaultdict(int)
ngrams_freq = {}
for tokens in train_set:
    for n in range(1, max_n+1):
        for i in range(len(tokens)-n+1):
            ngram = ' '.join(tokens[i:i+n])
            ngrams_counts[ngram] += 1

# Calculate the total frequency of each n-gram order
for n in range(1, max_n+1):
    ngrams_freq[str(n) + "_grams"] = sum(count for ngram, count in ngrams_counts.items() if len(ngram.split()) == n)


In [20]:
len(ngrams_counts)
ngrams_freq

{'1_grams': 354804, '2_grams': 329834, '3_grams': 304904, '4_grams': 279689}

In [21]:
ngrams_counts

defaultdict(int,
            {'<s>': 74391,
             'the': 14360,
             'project': 9,
             'gutenberg': 4,
             'ebook': 6,
             'of': 7948,
             'ulysses': 8,
             'by': 1243,
             'james': 32,
             'joyce': 3,
             '</s>': 24797,
             '<s> <s>': 49594,
             '<s> the': 1250,
             'the project': 4,
             'project gutenberg': 3,
             'gutenberg ebook': 2,
             'ebook of': 1,
             'of ulysses': 1,
             'ulysses by': 1,
             'by james': 6,
             'james joyce': 3,
             'joyce </s>': 3,
             '<s> <s> <s>': 24797,
             '<s> <s> the': 1250,
             '<s> the project': 1,
             'the project gutenberg': 3,
             'project gutenberg ebook': 2,
             'gutenberg ebook of': 1,
             'ebook of ulysses': 1,
             'of ulysses by': 1,
             'ulysses by james': 1,
             'by jam

In [22]:

n=4
d=0.75

In [23]:
def kn_prob(tokens,n , d):
    n_gram_str = str(n) + "_grams"
    
    def unigram_case(tokens):
        count = ngrams_counts.get(tokens[0], 0)
        return (count - d) / ngrams_freq[n_gram_str]
    
    def higher_order_ngram_case(tokens):
        ngram_word = ' '.join(tokens)
        prev_words = ' '.join(tokens[:-1])
        count = ngrams_counts.get(ngram_word, 0)
        first_term = max((count - d), 0) / ngrams_counts.get(prev_words, 0)
        lamda = (d / ngrams_freq[n_gram_str]) * ngrams_counts.get(ngram_word, 0)
        p_cont = kn_prob(tokens[:-1], n-1, d)
        return first_term + (lamda * p_cont)
    
    if len(tokens) <= 1:
        # Termination condition
        return 1
    elif n == 1:
        # Unigram case
        return unigram_case(tokens)
    else:
        # Higher order n-gram case
        return higher_order_ngram_case(tokens)



In [24]:

def kneser_ney(token_lists, n):
    d=0.75
    probability_list = []
    n_str = str(n) + "_grams"
    unigram_str = "1_grams"
    
    for tokens in token_lists:
        token_prob=[]
        for i in range(len(tokens)):
            prob = 1
        
            # Calculate unigram probability
            if (n == 1 or i == 0):
                word = tokens[i]
                
                
                if word not in ngrams_counts:
                    word = "<unk>"
                prob = (ngrams_counts[word] - d) / ngrams_freq[unigram_str]
            
            # Calculate n-gram probability
            
            elif i<n:
                ngram_word_tokens = tokens[:i+1]
                ngram_word = " ".join(ngram_word_tokens)
                prev_word_tokens = tokens[:i]
                prev_word = " ".join(prev_word_tokens)
                j = i
               
                # Back off to smaller n-grams
                while (ngram_word not in ngrams_counts) and j > 1:
                   
                    ngram_word = " ".join(tokens[j : i + 1])
                    prev_word = " ".join(tokens[j : i])
                    j-=1
                
                # Handle case where n-gram doesn't exist
                if ngram_word not in ngrams_counts or not prev_word:
                    word = "<unk>"
                    prob = (ngrams_counts[word] - d) / ngrams_freq[unigram_str]
                
                # Calculate n-gram probability using Kneser-Ney smoothing
                else:
                    first_term = max(ngrams_counts[ngram_word] - d, 0) / ngrams_counts[prev_word]
                    lambda_term = (d / ngrams_freq[n_str]) * ngrams_counts[ngram_word]
                    p_cont = kn_prob([tokens[:i]], n - 1, d)
                    prob = first_term + (lambda_term * p_cont)
            else:
                ngram_word = " ".join(tokens[i-n+1:i+1])
                prev_word = " ".join(tokens[i-n+1:i])
                j = i
                while (ngram_word not in ngrams_counts) and j>1:
                    ngram_word = " ".join(tokens[i-n+1:j])
                    prev_word = " ".join(tokens[i-n+1:j-1])
                    j -= 1
                
                if (ngram_word not in ngrams_counts) or prev_word=="":
                    word = '<unk>'
                    unigram_str = "1_grams"
                    prob = (ngrams_counts[ngram_word]-d) / ngrams_freq[unigram_str]
                
                else:
                    n_gram_str = str(n) + "_grams"
                    if ngrams_counts[prev_word] > 0:
                        first_term = max((ngrams_counts[ngram_word] - d), 0) / ngrams_counts[prev_word]
                    else:
                        first_term = 0
                    if ngrams_freq[n_gram_str] > 0:
                        lamda = (d/ngrams_freq[n_gram_str]) * ngrams_counts[ngram_word]
                    else:
                        lamda = 0
                    p_cont = kn_prob(prev_word.split(" "), n-1, d)
                    prob = first_term + (lamda * p_cont)
            token_prob.append(prob)
        probability_list.append(token_prob)
    return probability_list
                
                
   


In [25]:
train_prob_kn=kneser_ney(train_set,n)
test_prob_kn=kneser_ney(test_set,n)

In [26]:
test_prob_kn[0]




[0.20966575912334698,
 0.7996453687726924,
 0.5664792691882712,
 0.04773691953867487,
 0.7996453687726924,
 0.20966575912334698,
 4.2981477097214234e-05,
 0.05939687179389921,
 0.01757378721773148,
 1.7615359466071407e-05,
 0.0006461313852154992,
 0.0011168137901489273,
 0.0038345752618378385,
 0.0030514620376691916,
 0.0005199781676578161,
 1.4796901951499984e-05,
 0.0003501933461854996,
 0.0016083291446142777]

In [27]:
test_prob_kn[2]

[0.20966575912334698,
 0.7996453687726924,
 0.5664792691882712,
 0.003582511147589315,
 0.0018562734643060319,
 0.001344337174931332,
 0.005177924660649928,
 0.003458951984757782,
 6.341529407785707e-06,
 0.05921857096538241,
 0.00016472988636174297,
 1.1978444436928558e-05,
 0.6250005156827521,
 0.8500006063663333,
 0.11225188902582366,
 5.546871202647574e-05,
 8.807679733035704e-05,
 0.0005267905645393661]

In [28]:
import math

def perplexity_score(probs):
    perplexity = [math.pow(math.prod(sentence_prob), -1/len(sentence_prob)) for sentence_prob in probs]
    return perplexity

In [29]:

train_perplexity_kn= perplexity_score(train_prob_kn)
test_perplexity_kn=perplexity_score(test_prob_kn)


In [30]:
test_perplexity_kn_average=sum(test_perplexity_kn)/len(test_perplexity_kn)
train_perplexity_kn_average=sum(train_perplexity_kn)/len(train_perplexity_kn)

In [31]:
train_perplexity_kn_average


7.474226937093161

In [32]:

test_perplexity_kn_average

237.40171362621697

In [33]:
len(train_perplexity_kn)

24797

In [34]:
test_input=input("write sentence")

sub_data=tokenization(test_input)
sub_data=remove_punchu(sub_data)
sub_data=add_bracket(sub_data)
word=[split_text(sub_data)]


    


In [35]:
word

[['hello', 'bro']]

In [36]:
input_prob=kneser_ney(word,n)

In [37]:
input_prob[0]

[4.2981477097214234e-05, 0.04773691953867487]

In [38]:
input_perplexity=perplexity_score(input_prob)

In [39]:
input_perplexity

[698.123305846922]

In [40]:
def written_bell(token_lists, n_gram):
    """
    Calculates the Written-Bell probability of a list of token lists, given the n-gram size and beta factor.
    """
    beta=0.25
    prob_list = []
    for tokens in token_lists:
        token_prob=[]
        for i in range(len(tokens)):
            prob = 1
        
            # Calculate unigram probability
            if n_gram == 1 or i == 0:
                word = tokens[i]
                if word not in ngrams_counts:
                    word = "<unk>"
                prob = (ngrams_counts[word] + beta) / (ngrams_freq["1_grams"] + beta * len(ngrams_counts))
            
            # Calculate n-gram probability
            elif i < n_gram:
                ngram_word_tokens = tokens[:i+1]
                ngram_word = " ".join(ngram_word_tokens)
                prev_word_tokens = tokens[:i]
                prev_word = " ".join(prev_word_tokens)
                j = i
                
                # Back off to smaller n-grams
                while (ngram_word not in ngrams_counts) and j > 1:
                    ngram_word = " ".join(tokens[j : i + 1])
                    prev_word = " ".join(tokens[j : i])
                    j -= 1
                
                # Handle case where n-gram doesn't exist
                if ngram_word not in ngrams_counts or not prev_word:
                    word = "<unk>"
                    prob = (ngrams_counts[word] + beta) / (ngrams_freq["1_grams"] + beta * len(ngrams_counts))
                
                # Calculate n-gram probability using Written-Bell smoothing
                else:
                    first_term = ngrams_counts[ngram_word] + beta
                    
                    second_term = (1 - beta) * ngrams_counts[prev_word] / ngrams_counts.get(" ".join(ngram_word.split()[:-1]), 0)
                    prob = first_term / (ngrams_counts.get(tuple(ngram_word.split()[:-1]), 0) + beta * len(ngrams_counts)) + second_term

            else:
                ngram_word = " ".join(tokens[i-n_gram+1:i+1])
                prev_word = " ".join(tokens[i-n_gram+1:i])
                j = i
                while (ngram_word not in ngrams_counts) and j > 1:
                    ngram_word = " ".join(tokens[i-n_gram+1:j])
                    prev_word = " ".join(tokens[i-n_gram+1:j-1])
                    j -= 1
                
                if ngram_word not in ngrams_counts or not prev_word:
                    word = "<unk>"
                    prob = (ngrams_counts[word] + beta) / (ngrams_freq["1_grams"] + beta * len(ngrams_counts))
                
                else:
                    first_term = ngrams_counts[ngram_word] + beta
                    denominator = ngrams_counts.get(" ".join(ngram_word.split()[:-1]), 0)
                    if denominator == 0:
                        denominator = 1  # set denominator to a small value instead of 0
                    second_term = (1 - beta) * ngrams_counts[prev_word] / denominator
                    prob = first_term / (ngrams_counts.get(tuple(ngram_word.split()[:-1]), 0) + beta * len(ngrams_counts)) + second_term

            token_prob.append(prob)
        prob_list.append(token_prob)
    return prob_list


In [41]:
train_prob_wb=written_bell(train_set,n)
test_prob_wb= written_bell(test_set, n)

In [42]:
import math

def perplexity_score(probs):
    perplexity = [math.pow(math.prod(sentence_prob), -1/len(sentence_prob)) for sentence_prob in probs]
    return perplexity

In [43]:
train_perplexity_wb= perplexity_score(train_prob_wb)
test_perplexity_wb=perplexity_score(test_prob_wb)

In [44]:
test_perplexity_wb_average=sum(test_perplexity_wb)/len(test_perplexity_wb)
train_perplexity_wb_average=sum(train_perplexity_wb)/len(train_perplexity_wb)

In [45]:
test_perplexity_wb_average

4.485428917469285

In [46]:
train_perplexity_wb_average


1.4793007019276132

In [47]:
# with open("<2020101054>_LM_3_train_set.txt", "w") as f:
#     for i in range (len(train_set)):
#         line= " ".join(train_set[i]) +"\t"+str( train_perplexity_kn[i] )
#         f.write(line + "\n")
    

In [48]:
# with open("<2020101054>_LM_3_test_set.txt", "w") as f:
#     for i in range (len(test_set)):
#         line= " ".join(test_set[i]) +"\t"+str( test_perplexity_kn[i] )
#         f.write(line + "\n")
    

In [49]:
# with open("<2020101054>_LM_4_test_set.txt", "w") as f:
#     for i in range (len(test_set)):
#         line= " ".join(test_set[i]) +"\t"+str( test_perplexity_wb[i] )
#         f.write(line + "\n")
    

In [50]:
# with open("<2020101054>_LM_4_train_set.txt", "w") as f:
#     for i in range (len(train_set)):
#         line= " ".join(train_set[i]) +"\t"+str( train_perplexity_wb[i] )
#         f.write(line + "\n")